## Part 0 ##
Import necessary libraries and functions

In [1]:
from dataclasses import dataclass
from typing import *
import numpy as np
from scipy.stats import beta, chi2, poisson

## Part 2 ##
I decided to store information about relevance only, since for the all of the metrics from lecture 5 important is knowledge if document is relevant or not and the order of documents, which can be stored in one boolean array.

In [2]:
@dataclass
class Relevance:
    is_relevant: np.ndarray[np.bool_]


# I made 'kwargs' parameter to allow flexibility in choosing distribution
# for instance, using kwargs one can set a and b parameters for beta distribution (kwargs={'a':2, 'b':2} => beta.rvs(a=2, b=2))
def fake_search_result(distribution: Callable, kwargs: Dict, sample_size: int, relevance_percentage: float = 0.25) -> Relevance:
    is_relevant = np.random.rand(sample_size) < relevance_percentage
    is_relevant[-1] = True
    # here I use distribution to generate initial scores of fictional documents, get their sorted indices and rearrange relevance array according to them
    return Relevance(is_relevant=is_relevant[np.argsort(np.array(distribution(**kwargs, size=sample_size)))])

## Parts 1 and 3 ##
Ranking metrics of my choice are precision@k, F-beta@k, and RR(reciprocal rank).

In [3]:
class Ranking:
    # to create an instance of class Ranking one should provide an instance of class Relevance and the k value for each of the metrics
    def __init__(self, search_result: Relevance, k: int):
        self.search_result = search_result
        self.k = k

    # precision@k calculated as per slides: amount of relevant documents in first k documents / k
    def precision_k(self) -> float:
        return np.sum(self.search_result.is_relevant[:self.k]) / self.k

    # to calculate F-beta@k I also need recall@k
    # recall@k calculated as per slides: amount of relevant documents in first k documents / amount of all relevant documents
    def recall_k(self) -> float:
        return np.sum(self.search_result.is_relevant[:self.k]) / np.sum(self.search_result.is_relevant)

    # F-beta@k also is taken from slides, standard value of beta = 1 for convenience
    def fbeta_k(self, b: float = 1.0) -> float:
        precision = self.precision_k()
        recall = self.recall_k()
        # introduced condition to resolve cases with 0 precision and 0 recall leading to division by zero
        return (1 + b ** 2) * precision * recall / ((precision * b ** 2) + recall) if precision != 0 or recall != 0 else 0
        
    # RR from slides, np.where() helps to find first relevant document without loops
    def reciprocal_rank(self) -> float:
        rank = np.where(self.search_result.is_relevant)[0][0] + 1
        return 1 / rank

    def display_results(self) -> None:
        print("Relevance: ", self.search_result.is_relevant)
        print(f"Precision@{self.k}: {self.precision_k()}")
        print(f"Recall@{self.k}: {self.recall_k()}")
        print(f"F-beta@{self.k}: {self.fbeta_k()}")
        print(f"RR: {self.reciprocal_rank()}", end='\n\n')

## Part 4 ##
Evaluating my ranking metrics with different distributions (beta, chi-squared, and poisson)

In [4]:
# dict for info about distributions
d = {'distribution': [beta.rvs, chi2.rvs, poisson.rvs], 'kwargs': [{'a': 2, 'b': 2}, {'df': 4}, {'mu': 10}]}
for i in range(3):
    # name of the ditribution for reference
    print("Distribution: ", str(d['distribution'][i]).split('.')[-1].split('_')[0])
    # initialize an instance of Ranking class with different distributions and display the results
    Ranking(fake_search_result(distribution=d['distribution'][i], kwargs=d['kwargs'][i], sample_size=25), k=5).display_results()

Distribution:  beta
Relevance:  [False False  True False False  True False  True False  True False False
  True  True False  True False False False False False False  True False
 False]
Precision@5: 0.2
Recall@5: 0.125
F-beta@5: 0.15384615384615385
RR: 0.3333333333333333

Distribution:  chi2
Relevance:  [False  True False False False False  True  True False False  True False
 False False False False  True False False False False  True False False
 False]
Precision@5: 0.2
Recall@5: 0.16666666666666666
F-beta@5: 0.1818181818181818
RR: 0.5

Distribution:  poisson
Relevance:  [ True  True False  True False False False False False False False  True
 False False  True False  True False False False  True False False  True
 False]
Precision@5: 0.6
Recall@5: 0.375
F-beta@5: 0.4615384615384615
RR: 1.0

